Realisation of Compressed Row Storage matrix format

In [1]:
import copy

In [2]:
## values[] array contains non-zero values of the matrix
## p-s elements of col[] array contains the column index of the p-s value in values list
## rowIndex[] array contains index in the values[] array and indicates the first non-zero value that starts a row
class MatrixCRS():   #compressed row storage
    def __init__(self, matrix, zeros=False):      
        self.m = len(matrix)
        self.n = len(matrix[0])
        if not zeros:
            self.values, self.col, self.rowIndex = MatrixCRS.get_crs(matrix)
        else:
            self.values, self.col, self.rowIndex = [], [], [0 for _ in range(self.m+1)]
    def __getitem__(self, i):
        if type(i) == int:
            if i >= self.m:
                raise IndexError()
            row_i = [0 for i in range(self.n)]
            val_ind_begin = self.rowIndex[i]
            val_ind_end = self.rowIndex[i+1]
            for i in range(val_ind_begin, val_ind_end):
                row_i[self.col[i]] = self.values[i]
            return row_i
        else:
            j = i[1]
            i = i[0]
            if i >= self.m or j >= self.n:
                raise IndexError()
            val_ind_begin = self.rowIndex[i]
            val_ind_end = self.rowIndex[i+1]
            for i in range(val_ind_begin, val_ind_end):
                if self.col[i] == j:
                    return self.values[i]
            return 0  
    def __setitem__(self, i, value):
        if type(i) == int:
            if i >= self.m:
                raise IndexError()
        else:                
            j = i[1]
            i = i[0]
            if i >= self.m or j >= self.n:
                raise IndexError()
            val_ind_begin = self.rowIndex[i]
            val_ind_end = self.rowIndex[i+1]
            if val_ind_begin == val_ind_end:
                if value == 0:
                    return
                k = val_ind_begin-1
                self.values = self.values[:k+1] + [value] + self.values[k+1:]
                self.col = self.col[:k+1] + [j] + self.col[k+1:]
                for l in range(i+1, self.m+1):
                    self.rowIndex[l] += 1
                return
            for k in range(val_ind_begin, val_ind_end):                
                if self.col[k] == j:
                    if value == 0:
                        self.values = self.values[:k] + self.values[k+1:]
                        self.col = self.col[:k] + self.col[k+1:]
                        for l in range(i+1, self.m+1):
                            self.rowIndex[l] -= 1
                    else:
                        self.values[k] = value
                    return
                else:
                    if (k == val_ind_end - 1 or (k!=(val_ind_end-1) and self.col[k] < j and self.col[k+1] > j)):
                        if value == 0:
                            return
                        self.values = self.values[:k+1] + [value] + self.values[k+1:]
                        self.col = self.col[:k+1] + [j] + self.col[k+1:]
                        for l in range(i+1, self.m+1):
                            self.rowIndex[l] += 1
                        return      
    def __len__(self):
        return self.m
    ## static function
    def get_crs(mat): 
        m = len(mat)
        n = len(mat[0])
        val = []
        col = []
        rowId = []        
        for i in range(m):
            rowId+=[len(val)]
            for j in range(n):
                if mat[i][j] != 0:
                    val += [copy.deepcopy(mat[i][j])]
                    col += [j]
        rowId += [len(val)]
        return val, col, rowId
    def __str__(self):
        val_str = list(map(str, self.values))
        val_str = "values[] =\t[" + ', '.join(val_str) + "]"
        col_str = list(map(str, self.col))
        col_str = "columns[] =\t[" + ', '.join(col_str) + "]"
        row_str = list(map(str, self.rowIndex))
        row_str = "row_index[] =\t[" + ', '.join(row_str) + "]"
        out = '\n'.join([val_str, col_str, row_str])
        return out
    def __repr__(self):
        return str(self)

Example: print values

In [3]:
A = [[0, 0, 11, 3, 2],
     [4, -1, 3, 0, 0],
     [0, 0, 0, 0, 0], 
     [1, 0, 8, 7, 5], 
     [0, 0, 0, 9, 0]]
A_crs = MatrixCRS(A)

In [4]:
A_crs

values[] =	[11, 3, 2, 4, -1, 3, 1, 8, 7, 5, 9]
columns[] =	[2, 3, 4, 0, 1, 2, 0, 2, 3, 4, 3]
row_index[] =	[0, 3, 6, 6, 10, 11]

In [5]:
for i in range(A_crs.m):
    for j in range(A_crs.n):
        print(A_crs[i, j], end = " ")
    print()

0 0 11 3 2 
4 -1 3 0 0 
0 0 0 0 0 
1 0 8 7 5 
0 0 0 9 0 
